In [1]:
import mysql.connector
from pprint import pprint
import pandas as pd

db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'Nirmal9699',
    database = 'phonepe_pulse_data_visualization'
)

cur = db.cursor()
db.commit()

In [2]:
#agg_trans
def qust_1(s):
    sql = """SELECT state State, year Year, ROUND(sum(Transaction_Amount)) Transaction_Year 
        FROM phonepe_pulse_data_visualization.agg_trans  
        WHERE state = %s 
        GROUP BY year"""
    cur.execute(sql,(s,))
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State', 'Year','Transaction_Amount_By_Year']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_1('tamil-nadu'))

        State  Year Transaction_Amount_By_Year
1  tamil-nadu  2018               104019568191
2  tamil-nadu  2019               302562182998
3  tamil-nadu  2020               562710808076
4  tamil-nadu  2021              1321086760549
5  tamil-nadu  2022              2410492817244
6  tamil-nadu  2023              2325051489231


In [3]:
#agg_trans
def qust_2():
    sql = """SELECT state, SUM(Transaction_Count) AS Transaction_Count_By_State 
            FROM phonepe_pulse_data_visualization.agg_trans 
            GROUP BY state 
            ORDER BY Transaction_Count_By_State DESC 
            LIMIT 8"""
    cur.execute(sql)
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State', 'Transaction_Count_By_State']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_2())

            State Transaction_Count_By_State
1     maharashtra                16592144890
2       karnataka                15851869593
3       telangana                14124623312
4  andhra-pradesh                 9248341606
5       rajasthan                 8361862049
6   uttar-pradesh                 8205216641
7  madhya-pradesh                 7025112483
8           delhi                 4713503838


In [4]:
#agg_user
def qust_3(s,y):
    sql = """WITH SQ AS ( SELECT State,Quarter,Brand,ROUND(User_Percentage * 100) AS Percentage,
            ROW_NUMBER() OVER (PARTITION BY Quarter ORDER BY User_Percentage DESC) AS RowNum
            FROM phonepe_pulse_data_visualization.agg_user
            WHERE State = %s AND Year = %s)
            SELECT State,Quarter,Brand,Percentage
            FROM SQ
            WHERE RowNum <= 3;"""
    cur.execute(sql,(s,y))
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State', 'Quarter','Brand','Precentage']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_3('tamil-nadu',2021))

         State  Quarter    Brand Precentage
1   tamil-nadu        1   Xiaomi         23
2   tamil-nadu        1     Vivo         20
3   tamil-nadu        1  Samsung         19
4   tamil-nadu        2   Xiaomi         23
5   tamil-nadu        2     Vivo         20
6   tamil-nadu        2  Samsung         19
7   tamil-nadu        3   Xiaomi         23
8   tamil-nadu        3     Vivo         21
9   tamil-nadu        3  Samsung         18
10  tamil-nadu        4   Xiaomi         23
11  tamil-nadu        4     Vivo         22
12  tamil-nadu        4  Samsung         18


In [5]:
#map_trans
def qust_4(s,y):
    sql = """WITH SQ AS (
            SELECT State, Quarter, Year, Region, Amount,
            ROW_NUMBER() OVER (PARTITION BY Year, Quarter
            ORDER BY Amount DESC) AS RowNum
            FROM phonepe_pulse_data_visualization.map_trans
            WHERE State = %s
            )
            SELECT State, Quarter, Year, Region, Amount
            FROM SQ
            WHERE RowNum = 1 and YEAR = %s"""
    cur.execute(sql,(s,y))
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State', 'Quarter','Year','Region','Amount']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_4('telangana','2023'))

       State  Quarter  Year               Region           Amount
1  telangana        1  2023  rangareddy district  487359310308.00
2  telangana        2  2023  rangareddy district  561147925919.00
3  telangana        3  2023  rangareddy district  615864362548.00


In [6]:
#top_trans_dis
def qust_5(s):
    sql = """WITH SQ AS (
            SELECT State, Year, District, sum(Count) AS Count, sum(Amount) AS Amount,
            ROW_NUMBER() OVER (PARTITION BY State, Year ORDER BY sum(Amount) DESC) AS RowNum
            FROM phonepe_pulse_data_visualization.top_trans_dis 
            GROUP BY State,Year, District)
            SELECT State, District, Year, Count, Amount
            FROM SQ
            WHERE State = %s and RowNum = 1"""
    cur.execute(sql,(s,))
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State', 'District','Year','Count','Amount']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_5('west-bengal'))

         State                    District  Year      Count           Amount
1  west-bengal                     kolkata  2018   14679383   21628916825.87
2  west-bengal                     kolkata  2019   61291479   77520743688.39
3  west-bengal                     kolkata  2020   73618565  157860100499.32
4  west-bengal                     kolkata  2021  195758301  388417634006.16
5  west-bengal                     kolkata  2022  442825414  849910713054.79
6  west-bengal  north twenty four parganas  2023  292875219  452807293280.00


In [15]:
#map_user
def qust_6(s):
    sql = """WITH SQ AS (
            SELECT State, Year, Region, SUM(Count) AS Count,
            ROW_NUMBER() OVER (PARTITION BY Year ORDER BY SUM(Count)) AS RowNum
            FROM phonepe_pulse_data_visualization.map_user
            WHERE State = %s
            GROUP BY State, Year, Region)
            SELECT State, Year, RowNum, Region, Count
            FROM SQ 
            WHERE RowNum > 5 AND RowNum <= 8"""
    cur.execute(sql,(s,))
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State', 'Year','RowNum','Region','Count']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_6('tamil-nadu'))

         State  Year  RowNum                   Region   Count
1   tamil-nadu  2018       6         tenkasi district  109739
2   tamil-nadu  2018       7    nagapattinam district  115411
3   tamil-nadu  2018       8     pudukkottai district  118856
4   tamil-nadu  2019       6  ramanathapuram district  259492
5   tamil-nadu  2019       7    nagapattinam district  283392
6   tamil-nadu  2019       8     pudukkottai district  287686
7   tamil-nadu  2020       6  ramanathapuram district  477932
8   tamil-nadu  2020       7           karur district  512976
9   tamil-nadu  2020       8    nagapattinam district  515404
10  tamil-nadu  2021       6  ramanathapuram district  712155
11  tamil-nadu  2021       7           karur district  754321
12  tamil-nadu  2021       8    nagapattinam district  769669
13  tamil-nadu  2022       6  ramanathapuram district  909242
14  tamil-nadu  2022       7           karur district  963331
15  tamil-nadu  2022       8    nagapattinam district  979578
16  tami

In [19]:
#top_trans_pin
def qust_7():
    sql = """SELECT * FROM phonepe_pulse_data_visualization.top_trans_pin"""
    cur.execute(sql)
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State', 'Year','Quarter','Pincode','Amount','Count']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_7())

                          State  Year  Quarter Pincode         Amount    Count
1     andaman-&-nicobar-islands  2018        1  744101     2769297.90     1622
2     andaman-&-nicobar-islands  2018        1  744103     2238041.87     1223
3     andaman-&-nicobar-islands  2018        1  744102     3519059.94      969
4     andaman-&-nicobar-islands  2018        1  744105     1298560.95      685
5     andaman-&-nicobar-islands  2018        1  744104     1039715.31      340
...                         ...   ...      ...     ...            ...      ...
8206                west-bengal  2023        3  700001  7193521561.00  3536928
8207                west-bengal  2023        3  721301  4871566033.00  3464729
8208                west-bengal  2023        3  700039  5095824624.00  3405551
8209                west-bengal  2023        3  732125  5353831211.00  3347165
8210                west-bengal  2023        3  700015  5003673712.00  3304464

[8210 rows x 6 columns]


In [22]:
def qust_8():
    sql = """SELECT State, District, Round(Count)
            FROM phonepe_pulse_data_visualization.top_user_dis
            WHERE Year = 2023 and Quarter = 3
            ORDER BY Count DESC
            LIMIT 10"""
    cur.execute(sql)
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State', 'District','Count']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_8())

          State                    District     Count
1     karnataka             bengaluru urban  16382332
2   maharashtra                        pune  10780092
3   maharashtra                       thane   6802106
4     rajasthan                      jaipur   6745507
5   maharashtra             mumbai suburban   6560867
6     telangana                   hyderabad   5543537
7       gujarat                   ahmadabad   5105557
8     telangana                  rangareddy   4809474
9   west-bengal  north twenty four parganas   4241525
10      gujarat                       surat   4238878


In [25]:
def qust_9():
    sql = """WITH SQ as (
            SELECT State, Pincode, Round(Count) AS Count, Year, Quarter,
            ROW_NUMBER() OVER (PARTITION BY State ORDER BY Count) AS RowNum
            FROM phonepe_pulse_data_visualization.top_user_pin
            WHERE Year = 2023 and Quarter = 3)
            SELECT State, Pincode, Count
            FROM SQ
            WHERE RowNum = 1"""
    cur.execute(sql)
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State', 'Pincode','Count']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_9())

                                 State Pincode   Count
1            andaman-&-nicobar-islands  744205    2716
2                       andhra-pradesh  520001  186511
3                    arunachal-pradesh  791120   13685
4                                assam  781028   43026
5                                bihar  801503  156227
6                           chandigarh  160025   24439
7                         chhattisgarh  494001  104414
8   dadra-&-nagar-haveli-&-daman-&-diu  362570     508
9                                delhi  110042  413005
10                                 goa  403515   21511
11                             gujarat  382345  194466
12                             haryana  122004  283114
13                    himachal-pradesh  174303   32525
14                     jammu-&-kashmir  193224   27379
15                           jharkhand  832107   90713
16                           karnataka  560079  368803
17                              kerala  682032   40897
18        

In [26]:
def qust_10():
    sql = """SELECT State, ROUND((Sum(Amount)/Total_Amount),2)*100 AS Percentage
            FROM (SELECT Sum(Amount) AS Total_Amount 
            FROM phonepe_pulse_data_visualization.top_trans_dis
            WHERE Year = 2023) AS SQ
            JOIN phonepe_pulse_data_visualization.top_trans_dis AS M
            GROUP BY State, Total_Amount
            ORDER BY Percentage DESC"""
    cur.execute(sql)
    data = cur.fetchall()
    data_df = pd.DataFrame(data, columns=['State','Precentage']).reset_index(drop=True)
    data_df.index += 1
    return data_df

pprint(qust_10())

                                 State Precentage
1                            telangana      42.00
2                       andhra-pradesh      36.00
3                            karnataka      36.00
4                          maharashtra      35.00
5                            rajasthan      22.00
6                                delhi      15.00
7                       madhya-pradesh      14.00
8                        uttar-pradesh      14.00
9                          west-bengal      13.00
10                               bihar      11.00
11                             gujarat      10.00
12                              odisha      10.00
13                          tamil-nadu      10.00
14                             haryana       8.00
15                           jharkhand       5.00
16                        chhattisgarh       4.00
17                              kerala       4.00
18                               assam       3.00
19                              punjab       3.00
